In [7]:
# Pre-train용
import os
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
from sklearn.model_selection import train_test_split

class CustomBertForSequenceClassification(BertForSequenceClassification):
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        labels=None,
        output_hidden_states=True
    ):
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            labels=labels,
            output_hidden_states=output_hidden_states
        )
        logits = outputs.logits
        hidden_states = outputs.hidden_states[-5]  # n번째 레이어의 hidden states를 반환합니다.
        loss = outputs.loss
        return logits, loss, hidden_states

# 데이터 로드 및 전처리
data_A = pd.read_csv("output1.csv")  # data set A 파일명에 맞게 수정
data_B = pd.read_csv("infected.csv")  # data set B 파일명에 맞게 수정
# 모델 저장 경로
model_path = "Pre-trained.pt"

# X_train, Y_train 생성
X_train = []
Y_train = []

for index, row in data_A.iterrows():  # 중복 제거를 하지 않고 원본 데이터 사용
    patient_id = row["ID"]
    patient_info = [str(row[column]) for column in data_A.columns if column != "ID" and column != "DESCRIPTION"]
    symptoms = ", ".join(data_A[data_A["ID"] == patient_id]["DESCRIPTION"].tolist())
    combined_info = ", ".join(patient_info) + ", " + symptoms
    X_train.append(combined_info)
    if patient_id in data_B.values:
        Y_train.append(1)
    else:
        Y_train.append(0)

print("X_train\n", X_train[:10])
print("Y_train\n", Y_train[:10])
        
# BERT 토크나이저 및 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# 모델이 이미 저장되어 있는지 확인하고, 저장된 모델이 있으면 불러오고 없으면 새로운 모델 생성
if os.path.exists(model_path):
    # 저장된 모델이 있을 경우 불러오기
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    model.load_state_dict(torch.load(model_path))
    print("Pre-train model loaded.")
else:
    # 저장된 모델이 없을 경우 새로운 모델 생성
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    print("New model generated.")

# 입력 데이터를 BERT의 입력 형식으로 변환
max_len = 128  # 입력 시퀀스의 최대 길이

input_ids = []
attention_masks = []

for info in X_train:
    encoded_dict = tokenizer.encode_plus(
                        info,                         # 환자 정보 및 증상
                        add_special_tokens = True,    # [CLS], [SEP] 토큰 추가
                        max_length = max_len,         # 최대 길이 지정
                        pad_to_max_length = True,     # 패딩을 추가하여 최대 길이로 맞춤
                        return_attention_mask = True, # 어텐션 마스크 생성
                        return_tensors = 'pt',        # PyTorch 텐서로 반환
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(Y_train)

# 데이터셋 및 데이터로더 생성
dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = 0.8
train_dataset, val_dataset = train_test_split(dataset, test_size=1-train_size, random_state=42)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

# 모델을 GPU로 이동
model.to(device)

# 옵티마이저 및 학습률 설정
# 기본 학습률 : 2e-6
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-6)

# 에폭 설정
epochs = 3

# 학습 루프
hidden_states_list = []  # 모든 에폭에 대한 hidden state를 저장할 리스트
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs[1]  # loss가 outputs의 두 번째 값입니다.
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch + 1}/{epochs}, Batch Loss: {loss.item()}')
        # hidden state를 저장합니다.
        #hidden_states = outputs[2]
        #hidden_states_list.append(hidden_states)

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_train_loss}')

# 모든 에폭에 대한 hidden state를 합쳐서 CSV 파일로 저장합니다.
#hidden_states_concat = torch.cat(hidden_states_list, dim=0)
#hidden_states_concat = hidden_states_concat[:, 0, :].cpu().detach().numpy()
#hidden_states_df = pd.DataFrame(hidden_states_concat)
#hidden_states_df.to_csv("hidden_states_all_epochs.csv", index=False)

# 모델 저장
torch.save(model.state_dict(), model_path)

# 모델 평가
model.eval()
val_accuracy = 0
for batch in val_dataloader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'labels': batch[2]}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs[0]  # logits가 outputs의 첫 번째 값입니다.
    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    val_accuracy += (logits.argmax(axis=1) == label_ids).mean().item()

print(f'Validation Accuracy: {val_accuracy / len(val_dataloader)}')


X_train
 ["5/11/1967, nan, 999-83-3739, S99963041, X22481021X, Mrs., Hortense60, O'Hara248, nan, Spinka232, M, white, nonhispanic, F, Northborough  Massachusetts  US, 1040 Turner Knoll, Milford, Massachusetts, Worcester County, nan, 42.13519473, -71.50138606, 1137082.44, 9598.16, 11/12/2019, 1/21/2020, 803d9786-29a1-466c-9365-0205ea0a031c, 36971009.0, Sinusitis (disorder), Body mass index 30+ - obesity (finding), Prediabetes, Chronic sinusitis (disorder)", '11/19/1965, nan, 999-87-9895, S99998149, X88597197X, Mr., Waylon572, Reinger292, nan, nan, M, white, nonhispanic, M, Seekonk  Massachusetts  US, 895 Robel Light, Worcester, Massachusetts, Worcester County, 1604.0, 42.32627029, -71.79641283, 1134089.03, 5790.2, 5/7/2004, nan, ef7b5155-d4be-4b0f-98d4-1313f9b7e90b, 162864005.0, Body mass index 30+ - obesity (finding), Vomiting symptom (finding), Bacterial infectious disease (disorder), Pneumonia (disorder), Hypertension, Chronic sinusitis (disorder), Fever (finding), Hyperlipidemia, Hy

Some weights of CustomBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


New model generated.
True
Epoch 1/3, Batch Loss: 0.6847525238990784
Epoch 1/3, Batch Loss: 0.6722670793533325
Epoch 1/3, Batch Loss: 0.6310189366340637
Epoch 1/3, Batch Loss: 0.6308815479278564
Epoch 1/3, Batch Loss: 0.600823163986206
Epoch 1/3, Batch Loss: 0.6389783024787903
Epoch 1/3, Batch Loss: 0.5263268351554871
Epoch 1/3, Batch Loss: 0.6065617799758911
Epoch 1/3, Batch Loss: 0.5318933725357056
Epoch 1/3, Batch Loss: 0.6635839343070984
Epoch 1/3, Batch Loss: 0.5008132457733154
Epoch 1/3, Batch Loss: 0.48957595229148865
Epoch 1/3, Batch Loss: 0.4432156980037689
Epoch 1/3, Batch Loss: 0.5376770496368408
Epoch 1/3, Batch Loss: 0.5094747543334961
Epoch 1/3, Batch Loss: 0.6337817311286926
Epoch 1/3, Batch Loss: 0.38933855295181274
Epoch 1/3, Batch Loss: 0.4315186142921448
Epoch 1/3, Batch Loss: 0.44252073764801025
Epoch 1/3, Batch Loss: 0.4495561122894287
Epoch 1/3, Batch Loss: 0.3830277621746063
Epoch 1/3, Batch Loss: 0.6138479709625244
Epoch 1/3, Batch Loss: 0.6559330821037292
Epoch 

Epoch 1/3, Batch Loss: 0.2997588813304901
Epoch 1/3, Batch Loss: 0.4131537079811096
Epoch 1/3, Batch Loss: 0.35360416769981384
Epoch 1/3, Batch Loss: 0.3216302692890167
Epoch 1/3, Batch Loss: 0.2872118651866913
Epoch 1/3, Batch Loss: 0.28599852323532104
Epoch 1/3, Batch Loss: 0.44848763942718506
Epoch 1/3, Batch Loss: 0.4808341860771179
Epoch 1/3, Batch Loss: 0.13023139536380768
Epoch 1/3, Batch Loss: 0.26854580640792847
Epoch 1/3, Batch Loss: 0.21243765950202942
Epoch 1/3, Batch Loss: 0.274191290140152
Epoch 1/3, Batch Loss: 0.19439703226089478
Epoch 1/3, Batch Loss: 0.30279526114463806
Epoch 1/3, Batch Loss: 0.47825756669044495
Epoch 1/3, Batch Loss: 0.24582137167453766
Epoch 1/3, Batch Loss: 0.4181859493255615
Epoch 1/3, Batch Loss: 0.4626930058002472
Epoch 1/3, Batch Loss: 0.13820762932300568
Epoch 1/3, Batch Loss: 0.12077299505472183
Epoch 1/3, Batch Loss: 0.1602656990289688
Epoch 1/3, Batch Loss: 0.2628326117992401
Epoch 1/3, Batch Loss: 0.4668749272823334
Epoch 1/3, Batch Loss: 

Epoch 1/3, Batch Loss: 0.5151466131210327
Epoch 1/3, Batch Loss: 0.19452433288097382
Epoch 1/3, Batch Loss: 0.20477958023548126
Epoch 1/3, Batch Loss: 0.19719131290912628
Epoch 1/3, Batch Loss: 0.07704883068799973
Epoch 1/3, Batch Loss: 0.21937695145606995
Epoch 1/3, Batch Loss: 0.2202247530221939
Epoch 1/3, Batch Loss: 0.1811894178390503
Epoch 1/3, Batch Loss: 0.3259986340999603
Epoch 1/3, Batch Loss: 0.24670398235321045
Epoch 1/3, Batch Loss: 0.20213356614112854
Epoch 1/3, Batch Loss: 0.20732854306697845
Epoch 1/3, Batch Loss: 0.18254125118255615
Epoch 1/3, Batch Loss: 0.2918875813484192
Epoch 1/3, Batch Loss: 0.1025620549917221
Epoch 1/3, Batch Loss: 0.10840588808059692
Epoch 1/3, Batch Loss: 0.08697187900543213
Epoch 1/3, Batch Loss: 0.243134006857872
Epoch 1/3, Batch Loss: 0.22642536461353302
Epoch 1/3, Batch Loss: 0.18188968300819397
Epoch 1/3, Batch Loss: 0.09421464800834656
Epoch 1/3, Batch Loss: 0.1958257257938385
Epoch 1/3, Batch Loss: 0.18228699266910553
Epoch 1/3, Batch Los

...

Epoch 3/3, Batch Loss: 0.12827832996845245
Epoch 3/3, Batch Loss: 0.177622988820076
Epoch 3/3, Batch Loss: 0.03310970962047577
Epoch 3/3, Batch Loss: 0.0784197449684143
Epoch 3/3, Batch Loss: 0.39039915800094604
Epoch 3/3, Batch Loss: 0.47896531224250793
Epoch 3/3, Batch Loss: 0.18567229807376862
Epoch 3/3, Batch Loss: 0.09958261996507645
Epoch 3/3, Batch Loss: 0.1160217672586441
Epoch 3/3, Batch Loss: 0.16968859732151031
Epoch 3/3, Batch Loss: 0.20042477548122406
Epoch 3/3, Batch Loss: 0.01988375186920166
Epoch 3/3, Batch Loss: 0.04056260734796524
Epoch 3/3, Batch Loss: 0.07019909471273422
Epoch 3/3, Batch Loss: 0.4419177174568176
Epoch 3/3, Batch Loss: 0.0759660005569458
Epoch 3/3, Batch Loss: 0.09855903685092926
Epoch 3/3, Batch Loss: 0.07342477887868881
Epoch 3/3, Batch Loss: 0.07941167801618576
Epoch 3/3, Batch Loss: 0.04079863429069519
Epoch 3/3, Batch Loss: 0.1997780054807663
Epoch 3/3, Batch Loss: 0.07172474265098572
Epoch 3/3, Batch Loss: 0.0395829938352108
Epoch 3/3, Batch Lo

Epoch 3/3, Batch Loss: 0.16430127620697021
Epoch 3/3, Batch Loss: 0.1325996369123459
Epoch 3/3, Batch Loss: 0.19143006205558777
Epoch 3/3, Batch Loss: 0.20085807144641876
Epoch 3/3, Batch Loss: 0.19907377660274506
Epoch 3/3, Batch Loss: 0.27888059616088867
Epoch 3/3, Batch Loss: 0.2188510298728943
Epoch 3/3, Batch Loss: 0.03513288497924805
Epoch 3/3, Batch Loss: 0.11858030408620834
Epoch 3/3, Batch Loss: 0.3664565980434418
Epoch 3/3, Batch Loss: 0.1541798710823059
Epoch 3/3, Batch Loss: 0.09763067960739136
Epoch 3/3, Batch Loss: 0.09862413257360458
Epoch 3/3, Batch Loss: 0.20892837643623352
Epoch 3/3, Batch Loss: 0.12782315909862518
Epoch 3/3, Batch Loss: 0.09614238142967224
Epoch 3/3, Batch Loss: 0.17191259562969208
Epoch 3/3, Batch Loss: 0.0503455214202404
Epoch 3/3, Batch Loss: 0.08436547964811325
Epoch 3/3, Batch Loss: 0.20860685408115387
Epoch 3/3, Batch Loss: 0.37402036786079407
Epoch 3/3, Batch Loss: 0.038687434047460556
Epoch 3/3, Batch Loss: 0.06344728171825409
Epoch 3/3, Batc

Epoch 3/3, Batch Loss: 0.34711164236068726
Epoch 3/3, Batch Loss: 0.15629467368125916
Epoch 3/3, Batch Loss: 0.06774136424064636
Epoch 3/3, Batch Loss: 0.17540863156318665
Epoch 3/3, Batch Loss: 0.2309858202934265
Epoch 3/3, Batch Loss: 0.06871016323566437
Epoch 3/3, Batch Loss: 0.17036493122577667
Epoch 3/3, Batch Loss: 0.2876114845275879
Epoch 3/3, Batch Loss: 0.17037798464298248
Epoch 3/3, Batch Loss: 0.36086976528167725
Epoch 3/3, Batch Loss: 0.0929524376988411
Epoch 3/3, Batch Loss: 0.09265431016683578
Epoch 3/3, Batch Loss: 0.14028358459472656
Epoch 3/3, Batch Loss: 0.1486481875181198
Epoch 3/3, Batch Loss: 0.15818604826927185
Epoch 3/3, Batch Loss: 0.14402633905410767
Epoch 3/3, Batch Loss: 0.3162291646003723
Epoch 3/3, Batch Loss: 0.4148951768875122
Epoch 3/3, Batch Loss: 0.27749499678611755
Epoch 3/3, Batch Loss: 0.17490699887275696
Epoch 3/3, Batch Loss: 0.14623236656188965
Epoch 3/3, Batch Loss: 0.07228325307369232
Epoch 3/3, Batch Loss: 0.13195982575416565
Epoch 3/3, Batch 

Epoch 3/3, Batch Loss: 0.1679651141166687
Epoch 3/3, Batch Loss: 0.07242653518915176
Epoch 3/3, Batch Loss: 0.44177213311195374
Epoch 3/3, Batch Loss: 0.06924211978912354
Epoch 3/3, Batch Loss: 0.054864972829818726
Epoch 3/3, Batch Loss: 0.341724693775177
Epoch 3/3, Batch Loss: 0.2540881931781769
Epoch 3/3, Batch Loss: 0.2431134581565857
Epoch 3/3, Batch Loss: 0.14289723336696625
Epoch 3/3, Batch Loss: 0.20378534495830536
Epoch 3/3, Batch Loss: 0.6234591007232666
Epoch 3/3, Batch Loss: 0.1519833654165268
Epoch 3/3, Batch Loss: 0.09896190464496613
Epoch 3/3, Batch Loss: 0.0745716467499733
Epoch 3/3, Batch Loss: 0.12979517877101898
Epoch 3/3, Batch Loss: 0.1994321197271347
Epoch 3/3, Batch Loss: 0.04654661566019058
Epoch 3/3, Batch Loss: 0.14255373179912567
Epoch 3/3, Batch Loss: 0.08031487464904785
Epoch 3/3, Batch Loss: 0.15826934576034546
Epoch 3/3, Batch Loss: 0.224015012383461
Epoch 3/3, Batch Loss: 0.18246324360370636
Epoch 3/3, Batch Loss: 0.07443448901176453
Epoch 3/3, Batch Loss

Epoch 3/3, Batch Loss: 0.030734319239854813
Epoch 3/3, Batch Loss: 0.1911545991897583
Epoch 3/3, Batch Loss: 0.07540176808834076
Epoch 3/3, Batch Loss: 0.1665881872177124
Epoch 3/3, Batch Loss: 0.4484604001045227
Epoch 3/3, Batch Loss: 0.10307525098323822
Epoch 3/3, Batch Loss: 0.20333801209926605
Epoch 3/3, Batch Loss: 0.09012465178966522
Epoch 3/3, Batch Loss: 0.07120656222105026
Epoch 3/3, Batch Loss: 0.19247651100158691
Epoch 3/3, Batch Loss: 0.09406279027462006
Epoch 3/3, Batch Loss: 0.2052002102136612
Epoch 3/3, Batch Loss: 0.25156041979789734
Epoch 3/3, Batch Loss: 0.1411043405532837
Epoch 3/3, Batch Loss: 0.1604359745979309
Epoch 3/3, Batch Loss: 0.2366349697113037
Epoch 3/3, Batch Loss: 0.16271160542964935
Epoch 3/3, Batch Loss: 0.06695741415023804
Epoch 3/3, Batch Loss: 0.5400360822677612
Epoch 3/3, Batch Loss: 0.22339898347854614
Epoch 3/3, Batch Loss: 0.2953013777732849
Epoch 3/3, Batch Loss: 0.24248866736888885
Epoch 3/3, Batch Loss: 0.20203529298305511
Epoch 3/3, Batch Lo

In [8]:
# Fine-tune용
import os
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
from sklearn.model_selection import train_test_split

class CustomBertForSequenceClassification(BertForSequenceClassification):
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        labels=None,
        output_hidden_states=True
    ):
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            labels=labels,
            output_hidden_states=output_hidden_states
        )
        logits = outputs.logits
        hidden_states = outputs.hidden_states[-5]  # n번째 레이어의 hidden states를 반환합니다.
        loss = outputs.loss
        return logits, loss, hidden_states

# 데이터 로드 및 전처리
data_A = pd.read_csv("output3.csv")  # data set A 파일명에 맞게 수정
data_B = pd.read_csv("infected.csv")  # data set B 파일명에 맞게 수정
# 모델 불러오는 경로
model_path = "Pre-trained.pt"
# 모델 저장경로
model_path2 = "Fine-tuned.pt"

# X_train, Y_train 생성
X_train = []
Y_train = []

for index, row in data_A.iterrows():  # 중복 제거를 하지 않고 원본 데이터 사용
    patient_id = row["ID"]
    patient_info = [str(row[column]) for column in data_A.columns if column != "ID" and column != "DESCRIPTION"]
    symptoms = ", ".join(data_A[data_A["ID"] == patient_id]["DESCRIPTION"].tolist())
    combined_info = ", ".join(patient_info) + ", " + symptoms
    X_train.append(combined_info)
    if patient_id in data_B.values:
        Y_train.append(1)
    else:
        Y_train.append(0)

print("X_train\n", X_train[:10])
print("Y_train\n", Y_train[:10])
        
# BERT 토크나이저 및 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# 모델이 이미 저장되어 있는지 확인하고, 저장된 모델이 있으면 불러오고 없으면 새로운 모델 생성
if os.path.exists(model_path):
    # 저장된 모델이 있을 경우 불러오기
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    model.load_state_dict(torch.load(model_path))
    print("Pre-train model loaded.")
else:
    # 저장된 모델이 없을 경우 새로운 모델 생성
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    print("New model generated.")

# 입력 데이터를 BERT의 입력 형식으로 변환
max_len = 128  # 입력 시퀀스의 최대 길이

input_ids = []
attention_masks = []

for info in X_train:
    encoded_dict = tokenizer.encode_plus(
                        info,                         # 환자 정보 및 증상
                        add_special_tokens = True,    # [CLS], [SEP] 토큰 추가
                        max_length = max_len,         # 최대 길이 지정
                        pad_to_max_length = True,     # 패딩을 추가하여 최대 길이로 맞춤
                        return_attention_mask = True, # 어텐션 마스크 생성
                        return_tensors = 'pt',        # PyTorch 텐서로 반환
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(Y_train)

# 데이터셋 및 데이터로더 생성
dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = 0.8
train_dataset, val_dataset = train_test_split(dataset, test_size=1-train_size, random_state=42)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

# 모델을 GPU로 이동
model.to(device)

# 옵티마이저 및 학습률 설정
# 기본 학습률 : 2e-6
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-6)

# 에폭 설정
epochs = 3

# 학습 루프
hidden_states_list = []  # 모든 에폭에 대한 hidden state를 저장할 리스트
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs[1]  # loss가 outputs의 두 번째 값입니다.
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch + 1}/{epochs}, Batch Loss: {loss.item()}')
        # hidden state를 저장합니다.
        #hidden_states = outputs[2]
        #hidden_states_list.append(hidden_states)

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_train_loss}')

# 모든 에폭에 대한 hidden state를 합쳐서 CSV 파일로 저장합니다.
#hidden_states_concat = torch.cat(hidden_states_list, dim=0)
#hidden_states_concat = hidden_states_concat[:, 0, :].cpu().detach().numpy()
#hidden_states_df = pd.DataFrame(hidden_states_concat)
#hidden_states_df.to_csv("hidden_states_all_epochs.csv", index=False)

# 모델 저장
torch.save(model.state_dict(), model_path2)

# 모델 평가
model.eval()
val_accuracy = 0
for batch in val_dataloader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'labels': batch[2]}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs[0]  # logits가 outputs의 첫 번째 값입니다.
    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    val_accuracy += (logits.argmax(axis=1) == label_ids).mean().item()

print(f'Validation Accuracy: {val_accuracy / len(val_dataloader)}')


X_train
 ['5/11/1990, nan, 999-50-6899, S99965785, X23414632X, Mrs., Golden321, Berge125, nan, Becker968, M, white, nonhispanic, F, Saugus  Massachusetts  US, 900 Kihn Loaf Apt 75, Foxborough, Massachusetts, Norfolk County, nan, 42.07605525, -71.28353458, 645780.38, 7741.1, 5/10/2003, nan, 561d9445-df64-4fb9-a38a-37ccb670317e, 74400008.0, Appendicitis, Rupture of appendix', '10/5/1957, nan, 999-54-2768, S99932091, X75665632X, Mrs., Nevada145, Brakus656, nan, Schneider199, M, black, nonhispanic, F, Conway  Massachusetts  US, 844 Harris Ferry, Boston, Massachusetts, Suffolk County, 2130.0, 42.32797446, -71.15729167, 1352661.73, 14479.55, 3/8/2020, 3/16/2020, 92a64d19-1760-4344-b109-447bf15d70c1, 65710008.0, Acute respiratory failure (disorder), Hypoxemia (disorder)', '11/26/1969, nan, 999-84-2178, S99933880, X42152198X, Mrs., Tequila897, Ward668, nan, Larkin917, M, black, nonhispanic, F, Woburn  Massachusetts  US, 1083 Greenholt Lane, Athol, Massachusetts, Worcester County, 1331.0, 42.54

Some weights of CustomBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Pre-train model loaded.
True
Epoch 1/3, Batch Loss: 0.35644716024398804
Epoch 1/3, Batch Loss: 0.06446950137615204
Epoch 1/3, Batch Loss: 0.49225953221321106
Epoch 1/3, Batch Loss: 0.04072669893503189
Epoch 1/3, Batch Loss: 0.024975890293717384
Epoch 1/3, Batch Loss: 0.46829500794410706
Epoch 1/3, Batch Loss: 0.267522931098938
Epoch 1/3, Batch Loss: 0.05015619471669197
Epoch 1/3, Batch Loss: 0.31322547793388367
Epoch 1/3, Batch Loss: 0.11723592132329941
Epoch 1/3, Batch Loss: 0.027123499661684036
Epoch 1/3, Batch Loss: 0.31227344274520874
Epoch 1/3, Batch Loss: 0.1712123453617096
Epoch 1/3, Batch Loss: 0.1577252745628357
Epoch 1/3, Batch Loss: 0.09736476093530655
Epoch 1/3, Batch Loss: 0.1891738623380661
Epoch 1/3, Batch Loss: 0.28676292300224304
Epoch 1/3, Batch Loss: 0.08035378903150558
Epoch 1/3, Batch Loss: 0.20341376960277557
Epoch 1/3, Batch Loss: 0.14915987849235535
Epoch 1/3, Batch Loss: 0.2702379524707794
Epoch 1/3, Batch Loss: 0.07132475078105927
Epoch 1/3, Batch Loss: 0.2160

Epoch 1/3, Batch Loss: 0.1531742960214615
Epoch 1/3, Batch Loss: 0.2900690734386444
Epoch 1/3, Batch Loss: 0.07160258293151855
Epoch 1/3, Batch Loss: 0.032003406435251236
Epoch 1/3, Batch Loss: 0.2995308041572571
Epoch 1/3, Batch Loss: 0.19176769256591797
Epoch 1/3, Batch Loss: 0.39847245812416077
Epoch 1/3, Batch Loss: 0.0492219403386116
Epoch 1/3, Batch Loss: 0.17398422956466675
Epoch 1/3, Batch Loss: 0.13572709262371063
Epoch 1/3, Batch Loss: 0.12713825702667236
Epoch 1/3, Batch Loss: 0.22014327347278595
Epoch 1/3, Batch Loss: 0.5583798289299011
Epoch 1/3, Batch Loss: 0.0546497106552124
Epoch 1/3, Batch Loss: 0.20439910888671875
Epoch 1/3, Batch Loss: 0.24356591701507568
Epoch 1/3, Batch Loss: 0.0643753632903099
Epoch 1/3, Batch Loss: 0.3178096115589142
Epoch 1/3, Batch Loss: 0.14577288925647736
Epoch 1/3, Batch Loss: 0.16596296429634094
Epoch 1/3, Batch Loss: 0.20488418638706207
Epoch 1/3, Batch Loss: 0.09423104673624039
Epoch 1/3, Batch Loss: 0.08652830868959427
Epoch 1/3, Batch L

Epoch 1/3, Batch Loss: 0.34610921144485474
Epoch 1/3, Batch Loss: 0.29556795954704285
Epoch 1/3, Batch Loss: 0.4693663716316223
Epoch 1/3, Batch Loss: 0.03087153658270836
Epoch 1/3, Batch Loss: 0.29707369208335876
Epoch 1/3, Batch Loss: 0.09336701780557632
Epoch 1/3, Batch Loss: 0.2977447807788849
Epoch 1/3, Batch Loss: 0.2844799757003784
Epoch 1/3, Batch Loss: 0.16904765367507935
Epoch 1/3, Batch Loss: 0.1809263825416565
Epoch 1/3, Batch Loss: 0.09122513979673386
Epoch 1/3, Batch Loss: 0.14891384541988373
Epoch 1/3, Batch Loss: 0.17744840681552887
Epoch 1/3, Batch Loss: 0.03876589611172676
Epoch 1/3, Batch Loss: 0.09112913906574249
Epoch 1/3, Batch Loss: 0.14581064879894257
Epoch 1/3, Batch Loss: 0.047620125114917755
Epoch 1/3, Batch Loss: 0.16075921058654785
Epoch 1/3, Batch Loss: 0.03539394587278366
Epoch 1/3, Batch Loss: 0.03614191710948944
Epoch 1/3, Batch Loss: 0.06275549530982971
Epoch 1/3, Batch Loss: 0.3630457818508148
Epoch 1/3, Batch Loss: 0.556330144405365
Epoch 1/3, Batch 

...

Epoch 3/3, Batch Loss: 0.10885586589574814
Epoch 3/3, Batch Loss: 0.060866374522447586
Epoch 3/3, Batch Loss: 0.15056972205638885
Epoch 3/3, Batch Loss: 0.1282985806465149
Epoch 3/3, Batch Loss: 0.039345644414424896
Epoch 3/3, Batch Loss: 0.05962114408612251
Epoch 3/3, Batch Loss: 0.05160920321941376
Epoch 3/3, Batch Loss: 0.05373261496424675
Epoch 3/3, Batch Loss: 0.31569141149520874
Epoch 3/3, Batch Loss: 0.10887966305017471
Epoch 3/3, Batch Loss: 0.14469477534294128
Epoch 3/3, Batch Loss: 0.17081418633460999
Epoch 3/3, Batch Loss: 0.2547680735588074
Epoch 3/3, Batch Loss: 0.10032179206609726
Epoch 3/3, Batch Loss: 0.16850979626178741
Epoch 3/3, Batch Loss: 0.013749380595982075
Epoch 3/3, Batch Loss: 0.5373250246047974
Epoch 3/3, Batch Loss: 0.03562391549348831
Epoch 3/3, Batch Loss: 0.17635464668273926
Epoch 3/3, Batch Loss: 0.20212452113628387
Epoch 3/3, Batch Loss: 0.09952102601528168
Epoch 3/3, Batch Loss: 0.2747792899608612
Epoch 3/3, Batch Loss: 0.21777166426181793
Epoch 3/3, B

Epoch 3/3, Batch Loss: 0.19876965880393982
Epoch 3/3, Batch Loss: 0.14148783683776855
Epoch 3/3, Batch Loss: 0.19141295552253723
Epoch 3/3, Batch Loss: 0.027488749474287033
Epoch 3/3, Batch Loss: 0.14351306855678558
Epoch 3/3, Batch Loss: 0.059201184660196304
Epoch 3/3, Batch Loss: 0.25417789816856384
Epoch 3/3, Batch Loss: 0.2051084041595459
Epoch 3/3, Batch Loss: 0.43192559480667114
Epoch 3/3, Batch Loss: 0.15885265171527863
Epoch 3/3, Batch Loss: 0.40538695454597473
Epoch 3/3, Batch Loss: 0.15903152525424957
Epoch 3/3, Batch Loss: 0.24459466338157654
Epoch 3/3, Batch Loss: 0.2789527177810669
Epoch 3/3, Batch Loss: 0.1277133971452713
Epoch 3/3, Batch Loss: 0.12572315335273743
Epoch 3/3, Batch Loss: 0.04382966831326485
Epoch 3/3, Batch Loss: 0.04408558830618858
Epoch 3/3, Batch Loss: 0.09213494509458542
Epoch 3/3, Batch Loss: 0.3205047845840454
Epoch 3/3, Batch Loss: 0.2658361494541168
Epoch 3/3, Batch Loss: 0.2882544994354248
Epoch 3/3, Batch Loss: 0.09137771278619766
Epoch 3/3, Batc

Epoch 3/3, Batch Loss: 0.03962955251336098
Epoch 3/3, Batch Loss: 0.17252054810523987
Epoch 3/3, Batch Loss: 0.06631484627723694
Epoch 3/3, Batch Loss: 0.09252310544252396
Epoch 3/3, Batch Loss: 0.11239134520292282
Epoch 3/3, Batch Loss: 0.03258439153432846
Epoch 3/3, Batch Loss: 0.6044567227363586
Epoch 3/3, Batch Loss: 0.054372262209653854
Epoch 3/3, Batch Loss: 0.4064238369464874
Epoch 3/3, Batch Loss: 0.15903066098690033
Epoch 3/3, Batch Loss: 0.029791681095957756
Epoch 3/3, Batch Loss: 0.3152630031108856
Epoch 3/3, Batch Loss: 0.19722042977809906
Epoch 3/3, Batch Loss: 0.13422074913978577
Epoch 3/3, Batch Loss: 0.21994782984256744
Epoch 3/3, Batch Loss: 0.16267144680023193
Epoch 3/3, Batch Loss: 0.35829776525497437
Epoch 3/3, Batch Loss: 0.18595057725906372
Epoch 3/3, Batch Loss: 0.4097293019294739
Epoch 3/3, Batch Loss: 0.14947469532489777
Epoch 3/3, Batch Loss: 0.0685151219367981
Epoch 3/3, Batch Loss: 0.0975702628493309
Epoch 3/3, Batch Loss: 0.2135740965604782
Epoch 3/3, Batch

Epoch 3/3, Batch Loss: 0.06666618585586548
Epoch 3/3, Batch Loss: 0.16279500722885132
Epoch 3/3, Batch Loss: 0.22288434207439423
Epoch 3/3, Batch Loss: 0.10658922046422958
Epoch 3/3, Batch Loss: 0.1609991192817688
Epoch 3/3, Batch Loss: 0.392749160528183
Epoch 3/3, Batch Loss: 0.034985680133104324
Epoch 3/3, Batch Loss: 0.6328175663948059
Epoch 3/3, Batch Loss: 0.25472521781921387
Epoch 3/3, Batch Loss: 0.23559163510799408
Epoch 3/3, Batch Loss: 0.0516403503715992
Epoch 3/3, Batch Loss: 0.09910814464092255
Epoch 3/3, Batch Loss: 0.15503990650177002
Epoch 3/3, Batch Loss: 0.3975571393966675
Epoch 3/3, Batch Loss: 0.08343853056430817
Epoch 3/3, Batch Loss: 0.05380073934793472
Epoch 3/3, Batch Loss: 0.0813971757888794
Epoch 3/3, Batch Loss: 0.1504029631614685
Epoch 3/3, Batch Loss: 0.0378723219037056
Epoch 3/3, Batch Loss: 0.08079288899898529
Epoch 3/3, Batch Loss: 0.31293126940727234
Epoch 3/3, Batch Loss: 0.08859486132860184
Epoch 3/3, Batch Loss: 0.13454806804656982
Epoch 3/3, Batch Lo

Epoch 3/3, Batch Loss: 0.34209775924682617
Epoch 3/3, Batch Loss: 0.11648349463939667
Epoch 3/3, Batch Loss: 0.3485059440135956
Epoch 3/3, Batch Loss: 0.05554275959730148
Epoch 3/3, Batch Loss: 0.11406367272138596
Epoch 3/3, Batch Loss: 0.17429490387439728
Epoch 3/3, Batch Loss: 0.11334726214408875
Epoch 3/3, Batch Loss: 0.06822095066308975
Epoch 3/3, Batch Loss: 0.16231417655944824
Epoch 3/3, Batch Loss: 0.3176786005496979
Epoch 3/3, Batch Loss: 0.16107504069805145
Epoch 3/3, Batch Loss: 0.12196222692728043
Epoch 3/3, Batch Loss: 0.2595650255680084
Epoch 3/3, Batch Loss: 0.3519720137119293
Epoch 3/3, Batch Loss: 0.4072616994380951
Epoch 3/3, Batch Loss: 0.1399233639240265
Epoch 3/3, Batch Loss: 0.07074470818042755
Epoch 3/3, Batch Loss: 0.08141618221998215
Epoch 3/3, Batch Loss: 0.25972944498062134
Epoch 3/3, Batch Loss: 0.07537350058555603
Epoch 3/3, Batch Loss: 0.06127890571951866
Epoch 3/3, Batch Loss: 0.20548784732818604
Epoch 3/3, Batch Loss: 0.05908646807074547
Epoch 3/3, Batch 

In [15]:
# 데이터 랜덤분할(100/500)
import pandas as pd

def sample_csv_and_additional(input_file, output_file_500, output_file_100, n_500):
    # CSV 파일을 읽어옵니다.
    data = pd.read_csv(input_file)
    
    # 데이터를 랜덤하게 샘플링합니다.
    sampled_data_500 = data.sample(n=n_500, random_state=42)
    
    # 샘플링된 500개의 데이터를 CSV 파일로 내보냅니다.
    sampled_data_500.to_csv(output_file_500, index=False)
    
    # sampled_data_500에서 첫 100개의 데이터를 선택합니다.
    sampled_data_100 = sampled_data_500.head(100)
    
    # 선택된 첫 100개의 데이터를 CSV 파일로 내보냅니다.
    sampled_data_100.to_csv(output_file_100, index=False)

# 입력 CSV 파일 경로
input_file = "cleaned_covid_merged.csv"

# 출력 CSV 파일 경로
output_file_500 = "random_500.csv"
output_file_100 = "random_100.csv"

# 랜덤하게 추출할 데이터 개수
n_500 = 500

# 함수 호출
sample_csv_and_additional(input_file, output_file_500, output_file_100, n_500)


In [16]:
# smashed data 생성 (500/server side)
import os
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
from sklearn.model_selection import train_test_split

class CustomBertForSequenceClassification(BertForSequenceClassification):
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        labels=None,
        output_hidden_states=True
    ):
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            labels=labels,
            output_hidden_states=output_hidden_states
        )
        logits = outputs.logits
        hidden_states = outputs.hidden_states[-5]  # n번째 레이어의 hidden states를 반환합니다.
        loss = outputs.loss
        return logits, loss, hidden_states

# 데이터 로드 및 전처리
data_A = pd.read_csv("random_500.csv")  # data set A 파일명에 맞게 수정
data_B = pd.read_csv("infected.csv")  # data set B 파일명에 맞게 수정
# 모델 저장 경로
model_path = "Pre-trained.pt"

# X_train, Y_train 생성
X_train = []
Y_train = []

for index, row in data_A.iterrows():  # 중복 제거를 하지 않고 원본 데이터 사용
    patient_id = row["ID"]
    patient_info = [str(row[column]) for column in data_A.columns if column != "ID" and column != "DESCRIPTION"]
    symptoms = ", ".join(data_A[data_A["ID"] == patient_id]["DESCRIPTION"].tolist())
    combined_info = ", ".join(patient_info) + ", " + symptoms
    X_train.append(combined_info)
    if patient_id in data_B.values:
        Y_train.append(1)
    else:
        Y_train.append(0)

print("X_train\n", X_train[:10])
print("Y_train\n", Y_train[:10])
        
# BERT 토크나이저 및 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# 모델이 이미 저장되어 있는지 확인하고, 저장된 모델이 있으면 불러오고 없으면 새로운 모델 생성
if os.path.exists(model_path):
    # 저장된 모델이 있을 경우 불러오기
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    model.load_state_dict(torch.load(model_path))
    print("Pre-train model loaded.")
else:
    # 저장된 모델이 없을 경우 새로운 모델 생성
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    print("New model generated.")

# 입력 데이터를 BERT의 입력 형식으로 변환
max_len = 128  # 입력 시퀀스의 최대 길이

input_ids = []
attention_masks = []

for info in X_train:
    encoded_dict = tokenizer.encode_plus(
                        info,                         # 환자 정보 및 증상
                        add_special_tokens = True,    # [CLS], [SEP] 토큰 추가
                        max_length = max_len,         # 최대 길이 지정
                        pad_to_max_length = True,     # 패딩을 추가하여 최대 길이로 맞춤
                        return_attention_mask = True, # 어텐션 마스크 생성
                        return_tensors = 'pt',        # PyTorch 텐서로 반환
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(Y_train)

# 데이터셋 생성
dataset = TensorDataset(input_ids, attention_masks, labels)

# 데이터로더 생성
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

# 모델을 GPU로 이동
model.to(device)

# 모델 평가
model.eval()
val_accuracy = 0
hidden_states_list = []  # 평가할 때 hidden state를 저장할 리스트
for batch in dataloader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'labels': batch[2]}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs[0]  # logits가 outputs의 첫 번째 값입니다.
    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    val_accuracy += (logits.argmax(axis=1) == label_ids).mean().item()
    # hidden state를 저장합니다.
    hidden_states = outputs[2]
    hidden_states_list.append(hidden_states)
hidden_states_concat = torch.cat(hidden_states_list, dim=0)
hidden_states_concat = hidden_states_concat[:, 0, :].cpu().detach().numpy()
hidden_states_df = pd.DataFrame(hidden_states_concat)
hidden_states_df.to_csv("Dictionary_smashed_data.csv", index=False)

print(f'Validation Accuracy: {val_accuracy / len(dataloader)}')


X_train
 ["5/11/1967, nan, 999-83-3739, S99963041, X22481021X, Mrs., Hortense60, O'Hara248, nan, Spinka232, M, white, nonhispanic, F, Northborough  Massachusetts  US, 1040 Turner Knoll, Milford, Massachusetts, Worcester County, nan, 42.13519473, -71.50138606, 1137082.44, 9598.16, 11/12/2019, 1/21/2020, 803d9786-29a1-466c-9365-0205ea0a031c, 36971009.0, Sinusitis (disorder)", '11/19/1965, nan, 999-87-9895, S99998149, X88597197X, Mr., Waylon572, Reinger292, nan, nan, M, white, nonhispanic, M, Seekonk  Massachusetts  US, 895 Robel Light, Worcester, Massachusetts, Worcester County, 1604.0, 42.32627029, -71.79641283, 1134089.03, 5790.2, 5/7/2004, nan, ef7b5155-d4be-4b0f-98d4-1313f9b7e90b, 162864005.0, Body mass index 30+ - obesity (finding)', '1/20/1912, nan, 999-73-8598, S99979217, X22749225X, Mr., Hans694, Yost751, nan, nan, M, white, nonhispanic, M, Chelsea  Massachusetts  US, 158 Frami Drive Apt 55, Chelmsford, Massachusetts, Middlesex County, nan, 42.62575626, -71.41794612, 372357.04, 6

Some weights of CustomBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Pre-train model loaded.


C:\Users\MCC\anaconda3\envs\biotf\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


True
Validation Accuracy: 0.939453125


In [17]:
# smashed data 생성 (100/client side)
import os
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
from sklearn.model_selection import train_test_split

class CustomBertForSequenceClassification(BertForSequenceClassification):
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        labels=None,
        output_hidden_states=True
    ):
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            labels=labels,
            output_hidden_states=output_hidden_states
        )
        logits = outputs.logits
        hidden_states = outputs.hidden_states[-5]  # n번째 레이어의 hidden states를 반환합니다.
        loss = outputs.loss
        return logits, loss, hidden_states

# 데이터 로드 및 전처리
data_A = pd.read_csv("random_100.csv")  # data set A 파일명에 맞게 수정
data_B = pd.read_csv("infected.csv")  # data set B 파일명에 맞게 수정
# 모델 저장 경로
model_path = "Fine-tuned.pt"

# X_train, Y_train 생성
X_train = []
Y_train = []

for index, row in data_A.iterrows():  # 중복 제거를 하지 않고 원본 데이터 사용
    patient_id = row["ID"]
    patient_info = [str(row[column]) for column in data_A.columns if column != "ID" and column != "DESCRIPTION"]
    symptoms = ", ".join(data_A[data_A["ID"] == patient_id]["DESCRIPTION"].tolist())
    combined_info = ", ".join(patient_info) + ", " + symptoms
    X_train.append(combined_info)
    if patient_id in data_B.values:
        Y_train.append(1)
    else:
        Y_train.append(0)

print("X_train\n", X_train[:10])
print("Y_train\n", Y_train[:10])
        
# BERT 토크나이저 및 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# 모델이 이미 저장되어 있는지 확인하고, 저장된 모델이 있으면 불러오고 없으면 새로운 모델 생성
if os.path.exists(model_path):
    # 저장된 모델이 있을 경우 불러오기
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    model.load_state_dict(torch.load(model_path))
    print("Pre-train model loaded.")
else:
    # 저장된 모델이 없을 경우 새로운 모델 생성
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    print("New model generated.")

# 입력 데이터를 BERT의 입력 형식으로 변환
max_len = 128  # 입력 시퀀스의 최대 길이

input_ids = []
attention_masks = []

for info in X_train:
    encoded_dict = tokenizer.encode_plus(
                        info,                         # 환자 정보 및 증상
                        add_special_tokens = True,    # [CLS], [SEP] 토큰 추가
                        max_length = max_len,         # 최대 길이 지정
                        pad_to_max_length = True,     # 패딩을 추가하여 최대 길이로 맞춤
                        return_attention_mask = True, # 어텐션 마스크 생성
                        return_tensors = 'pt',        # PyTorch 텐서로 반환
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(Y_train)

# 데이터셋 생성
dataset = TensorDataset(input_ids, attention_masks, labels)

# 데이터로더 생성
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

# 모델을 GPU로 이동
model.to(device)

# 모델 평가
model.eval()
val_accuracy = 0
hidden_states_list = []  # 평가할 때 hidden state를 저장할 리스트
for batch in dataloader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'labels': batch[2]}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs[0]  # logits가 outputs의 첫 번째 값입니다.
    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    val_accuracy += (logits.argmax(axis=1) == label_ids).mean().item()
    # hidden state를 저장합니다.
    hidden_states = outputs[2]
    hidden_states_list.append(hidden_states)
hidden_states_concat = torch.cat(hidden_states_list, dim=0)
hidden_states_concat = hidden_states_concat[:, 0, :].cpu().detach().numpy()
hidden_states_df = pd.DataFrame(hidden_states_concat)
hidden_states_df.to_csv("Client_smashed_data.csv", index=False)

print(f'Validation Accuracy: {val_accuracy / len(dataloader)}')


X_train
 ["5/11/1967, nan, 999-83-3739, S99963041, X22481021X, Mrs., Hortense60, O'Hara248, nan, Spinka232, M, white, nonhispanic, F, Northborough  Massachusetts  US, 1040 Turner Knoll, Milford, Massachusetts, Worcester County, nan, 42.13519473, -71.50138606, 1137082.44, 9598.16, 11/12/2019, 1/21/2020, 803d9786-29a1-466c-9365-0205ea0a031c, 36971009.0, Sinusitis (disorder)", '11/19/1965, nan, 999-87-9895, S99998149, X88597197X, Mr., Waylon572, Reinger292, nan, nan, M, white, nonhispanic, M, Seekonk  Massachusetts  US, 895 Robel Light, Worcester, Massachusetts, Worcester County, 1604.0, 42.32627029, -71.79641283, 1134089.03, 5790.2, 5/7/2004, nan, ef7b5155-d4be-4b0f-98d4-1313f9b7e90b, 162864005.0, Body mass index 30+ - obesity (finding)', '1/20/1912, nan, 999-73-8598, S99979217, X22749225X, Mr., Hans694, Yost751, nan, nan, M, white, nonhispanic, M, Chelsea  Massachusetts  US, 158 Frami Drive Apt 55, Chelmsford, Massachusetts, Middlesex County, nan, 42.62575626, -71.41794612, 372357.04, 6

Some weights of CustomBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Pre-train model loaded.
True
Validation Accuracy: 0.9107142857142857


In [25]:
# Top N NN산출
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def read_csv_data(file_paths):
    data = []
    for file_path in file_paths:
        df = pd.read_csv(file_path, header=None)
        data.append(df.values)
    return data

def calculate_similarity(client_data, server_data):
    similarity_scores = cosine_similarity(client_data, server_data)
    return similarity_scores

def get_top_n_inferences(similarity_scores, n):
    top_n_indices = np.argsort(similarity_scores, axis=1)[:, -n:]
    return top_n_indices

def main(client_files, server_files, n):
    # Read CSV data
    client_data = read_csv_data(client_files)
    server_data = read_csv_data(server_files)

    # Exclude the first row from server data for comparison
    server_data = [data[1:] for data in server_data]

    # Convert data to numpy arrays
    client_data = np.vstack(client_data)
    server_data = np.vstack(server_data)

    # Calculate similarity
    similarity_scores = calculate_similarity(client_data, server_data)

    # Get top N inferences
    top_n_indices = get_top_n_inferences(similarity_scores, n)

    # Output top N inferences
    for i, indices in enumerate(top_n_indices):
        print(f"Top {n} inferences for client {i}:")
        for idx in indices:
            print(f"Server {idx} with similarity score {similarity_scores[i][idx]}")

if __name__ == "__main__":
    client_files = ['Client_smashed_data.csv']
    server_files = ['Dictionary_smashed_data.csv']
    n = 5  # Number of top inferences
    main(client_files, server_files, n)


Top 5 inferences for client 0:
Server 69 with similarity score -0.027380570578298798
Server 141 with similarity score -0.027331109428647766
Server 289 with similarity score -0.02709142236258632
Server 151 with similarity score -0.025491445245368997
Server 247 with similarity score -0.02339054532210398
Top 5 inferences for client 1:
Server 431 with similarity score 0.9670944617132706
Server 358 with similarity score 0.9694984380883578
Server 153 with similarity score 0.9712645401455625
Server 195 with similarity score 0.9721232025845163
Server 279 with similarity score 0.9853076338775929
Top 5 inferences for client 2:
Server 452 with similarity score 0.9693298776947514
Server 87 with similarity score 0.96933228996485
Server 8 with similarity score 0.9695756022357571
Server 432 with similarity score 0.9696847321832198
Server 269 with similarity score 0.9757509783407778
Top 5 inferences for client 3:
Server 413 with similarity score 0.9721511980602019
Server 207 with similarity score 0.97

In [28]:
# Top@N Accuracy 산출
import pandas as pd
import numpy as np

def euclidean_distance(vector1, vector2):
    return np.linalg.norm(vector1 - vector2)

def calculate_top_n_accuracy(client_data, dictionary_data, n):
    success_count = 0
    success_indices = []
    for i in range(len(client_data)):
        client_vector = np.array(client_data.iloc[i])
        distances = []
        for j in range(1, len(dictionary_data)):  # 인덱스 0을 제외하고 비교
            dictionary_vector = np.array(dictionary_data.iloc[j])
            distance = euclidean_distance(client_vector, dictionary_vector)
            distances.append((distance, j))  # (distance, index)
        distances.sort()  # Sort distances in ascending order
        top_n_indices = [index for _, index in distances[:n]]  # Get top N indices
        if i+1 in top_n_indices:  # client 데이터 인덱스를 1부터 시작하므로 i+1 사용
            success_count += 1
            success_indices.append(i+1)  # client 데이터 인덱스를 1부터 시작하므로 i+1 사용
    return success_count / len(client_data), success_indices

# 두 암호화된 파일 로드
client_data = pd.read_csv("Client_smashed_data.csv", header=None)
dictionary_data = pd.read_csv("Dictionary_smashed_data.csv", header=None)

# 상위 N개 설정
top_n = 5

# Top@N 정확도 계산
accuracy, success_indices = calculate_top_n_accuracy(client_data, dictionary_data, top_n)
print(f"Top@{top_n} Accuracy: {accuracy}")
print(f"Success Indices: {success_indices}")


Top@5 Accuracy: 0.009900990099009901
Success Indices: [85]
